In [ ]:
# Author: Victoria Yan, EMBL OSIM 
# Date June 2022 
# Motivation is to parse and distinguish country and institute information for EMBL publications
# Data is sourced from WoS's export of 2019-2022 publication data
# Code could be adapted to change the institute name to another organization to find your collaborating countries

import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from pylab import *
from matplotlib.colors import *
from pathlib import Path  
import re

In [ ]:
# Load data
dataPath = os.path.abspath('/Users/yan/Documents/Python/EMBL_DataCollection/data/2019-2022_fullWoS - Sheet2.tsv')
WoS_tab = pd.read_csv(dataPath, sep='\t')

In [ ]:
WoS_tab
sepStart=[]

In [ ]:
# define parsing function
# This parses the paragraph of addresses down to a dataframe with column Name, institute, Country, 
# each row is another instance of institute

def parse_affiliations(address):
    sepStart = cPub.split("; [")
    dfAffil = pd.DataFrame(columns=['Names','Institute','Country'])
    ll = len(sepStart)
    
    for i in range(0,ll):
        currentAffil = sepStart[i]
        isolateName = currentAffil.split("]")
        Name = isolateName[0].split("[")
        isolateCountry = isolateName[1].split()
        if i == 0:
            dfAffil.loc[i, ["Names"]] = str(Name[1])
        else:
            dfAffil.loc[i, ["Names"]] = str(Name[0])
        
        dfAffil.loc[i, ["Institute"]] = isolateName[1]
        dfAffil.loc[i, ["Country"]] = isolateCountry[-1]  
    return dfAffil

In [ ]:
# loop through all publications in table
# get affiliation table, then extract unique collaborating countries

p = len(WoS_tab)

for pb in range(1,p):
    print(pb)
    cPub = WoS_tab.iloc[pb,1]
    result = parse_affiliations(cPub)
    
    # Identify the EMBL authors in the table
    # Possible to change institute here by replacing this string
    FindEmblAuthor = result[result['Institute'].str.contains('European Mol Biol Lab')]
    
    if len(FindEmblAuthor) == 0:
        FindEmblAuthor = result[result['Institute'].str.contains('EMBL')]
    
    if len(FindEmblAuthor) == 0:
        FindEmblAuthor = result[result['Institute'].str.contains('European Bioinformat Inst')]
    
    if len(FindEmblAuthor) == 0:
        FindEmblAuthor = result[result['Institute'].str.contains('HGNC')]
      
    EMBLAuthorName = FindEmblAuthor.iloc[0]["Names"]
    substring = ";"
    
    # Check if there are multiple EMBL authors
    if substring in EMBLAuthorName:
        separateNames = EMBLAuthorName.split(";")
    else: 
        separateNames = [EMBLAuthorName]
    
    for k in range(0,len(separateNames)):  
        separateNames[k] = separateNames[k].lstrip()
        
    nonEMBLAuthorTab = pd.DataFrame(columns=['Names','Institute','Country'])

    for l in range(0,len(result)):
        cRow = result.iloc[l]
        rowNames = str(cRow['Names'])
        
        boolAlloc = []
        
        for m in range(0, len(separateNames)):
        
            cName = separateNames[m]
        
            if cName in rowNames:
                check1 = True
            
            else: 
                check1 = False    
            
            boolAlloc.append(check1) 
        
        if not any(boolAlloc):
            nonEMBLAuthorTab = nonEMBLAuthorTab.append(cRow, ignore_index=True)
        else:
            pass
    
        nonEMBLAuthorTab.drop_duplicates(keep='last', subset = None)
        uniqueCollabCountries = (nonEMBLAuthorTab["Country"].unique())
        
    if len(uniqueCollabCountries) > 0:
        
        print(uniqueCollabCountries)
        WoS_tab.loc[pb, ["UniqueCollabCountries"]] = str(uniqueCollabCountries)
        #WoS_tab[pb]=[WoS_tab[pb][0],WoS_tab[pb][1],uniqueCollabCountries]
        

In [ ]:
#filepath = Path('/Users/yan/Documents/Python/EMBL_DataCollection/data/2021_WoSAffil_res1.tsv') 
WoS_tab.to_csv('2021_WoSAffil_analysis2.tsv', sep = "\t")